In [1]:
import pandas as pd
import numpy as np

listingsDF = pd.read_csv('listings.csv')

calendarDF = pd.read_csv('calendar.csv')

reviewsDF = pd.read_csv('reviews.csv')

In [10]:
#We clean reviews first
#reviewsDF.info()
#reviewsDF.head()

#We need to remove '(email hidden)', special symbols
reviewsDF = reviewsDF.loc[reviewsDF['reviewer_name'].str.contains(r'[^\x00-\x7F]+') == False]

reviewsDF = reviewsDF.loc[reviewsDF['reviewer_name'].str.contains('email hidden') == False]

#Then replace '()', '-'
reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('[()]', '')

reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('[-]', ' ')

#Then replace 'And' with '&'
reviewsDF.reviewer_name = reviewsDF.reviewer_name.str.replace('And', '&')

#Now we remove all rows with no reviews
reviewsDF = reviewsDF.dropna(axis=0, how='any')

#Check for row
#reviewsDF.loc[reviewsDF['reviewer_id'] == 12120141]

#reviewsDF.info()

,listing_id,id,date,reviewer_id,reviewer_name,comments


In [13]:
#Now we clean listings

#Instead of dropping we select what we need
listingsDF = listingsDF[['id', 'name', 'summary', 'space', 'description', 
                         'host_id', 'host_name', 'latitude', 'longitude', 'property_type', 'price', 
                         'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']]

#We drop all rows with empty cells
listingsDF = listingsDF.dropna(axis=0, how='any')

#listingsDF.info()

#Remove the '$' from price
listingsDF.price = listingsDF.price.str.replace('[$]', '')

#Replace host name 'And' with '&'
listingsDF.price = listingsDF.price.str.replace('And', '&')

#listingsDF.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2668 entries, 1 to 3814
Data columns (total 19 columns):
id                             2668 non-null int64
name                           2668 non-null object
summary                        2668 non-null object
space                          2668 non-null object
description                    2668 non-null object
host_id                        2668 non-null int64
host_name                      2668 non-null object
latitude                       2668 non-null float64
longitude                      2668 non-null float64
property_type                  2668 non-null object
price                          2668 non-null object
number_of_reviews              2668 non-null int64
review_scores_rating           2668 non-null float64
review_scores_accuracy         2668 non-null float64
review_scores_cleanliness      2668 non-null float64
review_scores_checkin          2668 non-null float64
review_scores_communication    2668 non-null float64
revie

,id,name,summary,space,description,host_id,host_name,latitude,longitude,property_type,price,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
1,953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,5177328,Andrea,47.639123,-122.365666,Apartment,150.00,43,96.0,10.0,10.0,10.0,10.0,10.0,10.0
2,3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,16708587,Jill,47.629724,-122.369483,House,975.00,20,97.0,10.0,10.0,10.0,10.0,10.0,10.0
4,278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,1452570,Emily,47.632918,-122.372471,House,450.00,38,92.0,9.0,9.0,10.0,10.0,9.0,9.0
5,5956968,Private unit in a 1920s mansion,We're renting out a small private unit of one ...,If you include a bit of your background in you...,We're renting out a small private unit of one ...,326758,Andrew,47.630525,-122.366174,House,120.00,17,95.0,10.0,10.0,10.0,10.0,10.0,10.0
6,1909058,Queen Anne Private Bed and Bath,Enjoy a quiet stay in our comfortable 1915 Cra...,Enjoy a quiet stay in our comfortable 1915 Cra...,Enjoy a quiet stay in our comfortable 1915 Cra...,2497928,Gary And Beth,47.636605,-122.368519,House,80.00,58,99.0,10.0,10.0,10.0,10.0,10.0,10.0
